In [1]:
from kerax.layers import Conv2D, Dense, Flatten, Input, Activation, Add, Concatenate
from kerax.models import Model
from jax.random import PRNGKey
from jax import random
from jax import numpy as jnp

import numpy as np
import itertools
from collections import OrderedDict
import jax

2022-04-04 05:30:07.413000: E external/org_tensorflow/tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [2]:
img = random.normal(key=PRNGKey(3), shape=(2, 28, 28, 1))

# train_x = random.normal(key=PRNGKey(3), shape=(128, 28, 28, 1))
# train_y = random.randint(key=PRNGKey(34), shape=(128, 1), minval=0, maxval=1)

In [3]:
inputs = Input(shape=(28, 28, 1))
conv1 = Conv2D(64, 3, key=PRNGKey(100), padding='same')(inputs)
act1 = Activation('leaky_relu')(conv1)
conv3 = Conv2D(128, 3, padding='same', key=PRNGKey(104))(act1)
act2 = Activation('leaky_relu')(conv3)
conv4 = Conv2D(128, 3, padding='same', key=PRNGKey(105))(act2)
# add1 = Add()([conv4, act2])

# inputs2 = Input(shape=(28, 28, 1))
# inputs2_conv1 = Conv2D(32, 3, key=PRNGKey(100), padding='same')(inputs2)
# inputs2_act1 = Activation('relu')(inputs2_conv1)
# inputs2_conv3 = Conv2D(128, 3, padding='same', key=PRNGKey(104))(inputs2_act1)
# inputs2_act2 = Activation('gelu')(inputs2_conv3)
# inputs2_conv4 = Conv2D(128, 3, padding='same', key=PRNGKey(105))(inputs2_act2)
# add2 = Add()([inputs2_conv4, inputs2_act2])


# inputs3 = Input(shape=(28, 28, 1))
# inputs3_conv1 = Conv2D(32, 3, key=PRNGKey(100), padding='same')(inputs3)
# inputs3_act1 = Activation('relu')(inputs3_conv1)
# inputs3_conv3 = Conv2D(128, 3, padding='same', key=PRNGKey(104))(inputs3_act1)
# inputs3_act2 = Activation('gelu')(inputs3_conv3)
# inputs3_conv4 = Conv2D(128, 3, padding='same', key=PRNGKey(104))(inputs3_act2)
# inputs3_conv5 = Conv2D(128, 3, padding='same', key=PRNGKey(105))(inputs3_conv4)
# add3 = Add()([inputs3_conv5, inputs3_act2])

# cat = Concatenate()([add1, add2, add3])

flatten1 = Flatten()(conv4)
flatten2 = Flatten()(act2)
dense1 = Dense(512, activation='leaky_relu')(flatten1)
dense2 = Dense(512)(flatten2)
output = Dense(1, activation='sigmoid')(dense1)
output2 = Dense(1)(dense2)

In [11]:
def traverse(inputs):
    branches = [OrderedDict([(_input.name, _input)]) for _input in inputs]
    main_layers = OrderedDict()
    def _traverse(root, branch_index, visited):
        nonlocal branches
        for r in root._node_container.outbound_nodes:
            if r.name not in visited:
                visited.add(r.name)
                if branch_index > 0 and r.name in branches[branch_index-1] and r.name not in main_layers:
                    del(branches[branch_index-1][r.name])
                    main_layers[r.name] = r
                elif r.name in main_layers:
                    continue
                else:
                    branches[branch_index][r.name] = r
                _traverse(r, branch_index, visited)

    for branch_index, _input in enumerate(inputs, start=0):
        _traverse(root=_input, branch_index=branch_index, visited=set())

    def create_graph():
        nonlocal branches
        requires_output_from = OrderedDict()
        requires_output_to = OrderedDict()
        layers = OrderedDict([(layer_name, layer) for branch in branches for layer_name, layer in branch.items()])

        layers.update(main_layers)
        for layer_name, layer in layers.items():
            required_outputs_from = [node.name for node in layer._node_container.inbound_nodes]
            required_outputs_to = [node.name for node in layer._node_container.outbound_nodes]
            requires_output_from[layer_name] = required_outputs_from
            requires_output_to[layer_name] = required_outputs_to


        return layers, requires_output_from, requires_output_to

    layers, requires_output_from, requires_output_to = create_graph()

    params = []
    for l_name, l_param in requires_output_from.items():
        params.append(layers[l_name].params)

    params = tuple(params)

    def flow_input(params, tensors):
        saved_outputs = OrderedDict()
        nonlocal layers, requires_output_from, requires_output_to
        consumed_indices = 0
        for param, (key, val) in zip(params, requires_output_from.items()):
            if len(val) == 0:
                saved_outputs[key] = layers[key].call_with_external_weights(param, tensors[consumed_indices])
                consumed_indices += 1
            else:
                if len(val) == 1:
                    saved_outputs[key] = layers[key].call_with_external_weights(param, saved_outputs[val[0]])
                else:
                    saved_outputs[key] = layers[key].call_with_external_weights(param, [saved_outputs[v] for v in val])

        return saved_outputs

    
    # x = np.random.random(((1, 28, 28, 1)))

    y = flow_input(params, [img])

    return branches, layers, requires_output_from, requires_output_to, y


In [12]:
branches = traverse([inputs])

In [14]:
branches[-1].keys()

odict_keys(['Input_1', 'Conv2D_1', 'Activation_1', 'Conv2D_2', 'Activation_2', 'Conv2D_3', 'Flatten_1', 'Dense_1', 'Dense_3', 'Flatten_2', 'Dense_2', 'Dense_4'])

In [95]:
params

NameError: name 'params' is not defined

In [91]:
class GraphV2:
    allowed_kwargs = {'input', 'inputs', 'output', 'outputs', 'save_branches'}
    def __init__(self, *args, **kwargs):
        self.inputs, self.outputs = self._check_args_and_kwargs(*args, **kwargs)
        branches, self.multiple_branches = self.get_branches()
        self.layers, self.dependencies, self.params = self.create_graph(branches)

    def flatten(self, inputs):
        if isinstance(inputs, (list, tuple)):
            return list(itertools.chain(inputs))
        else:
            return [inputs]
    
    def _check_args_and_kwargs(self, *args, **kwargs):
        inputs, outputs = self._check_args(*args)

        if not inputs and not outputs:
            inputs, outputs = self._check_kwargs(**kwargs)
            if not inputs and not outputs:
                raise Exception('Cannot create a Model without inputs or outputs')
            else:
                return inputs, outputs
        else:
            return inputs, outputs


    def _check_args(self, *args):
        if len(args) == 0:
            return None, None
        inputs = self.flatten(args[0])
        outputs = self.flatten(args[1])
        return inputs, outputs


    def _check_kwargs(self, **kwargs):
        if len(kwargs) == 0:
            return None, None
        
        for k, v in kwargs.items():
            if k not in GraphV2.allowed_kwargs:
                raise Exception(f'Unknown argument {k}')
            else:
                if k == 'input' or k == 'inputs':
                    if k == 'input' and isinstance(v, (list, tuple)):
                        raise Exception(f'Expected a single element found an element in a {type(v)}')
                    elif k == 'inputs' and not isinstance(v, (list, tuple)):
                        raise Exception(f'Expected the element to be found in a list or a tuple found {type(v)}')
                    else:
                        inputs = self.flatten(v)
                    
                elif k == 'output' or k == 'outputs':
                    if k == 'output' and isinstance(v, (list, tuple)):
                        raise Exception(f'Expected a single element found an element in a {type(v)}')
                    elif k == 'outputs' and not isinstance(v, (list, tuple)):
                        raise Exception(f'Expected the element to be found in a list or a tuple found {type(v)}')
                    else:
                        outputs = self.flatten(v)
        return inputs, outputs


    def get_branches(self):
        branches = [OrderedDict([(_input.name, _input)]) for _input in self.inputs]
        multiple_branches = len(branches) > 1

        main_layers = OrderedDict()
        def _traverse(root, branch_index, visited):
            nonlocal branches
            for r in root._node_container.outbound_nodes:
                if r.name not in visited:
                    visited.add(r.name)
                    if branch_index > 0 and r.name in branches[branch_index-1] and r.name not in main_layers:
                        del(branches[branch_index-1][r.name])
                        main_layers[r.name] = r
                    elif r.name in main_layers:
                        continue
                    else:
                        branches[branch_index][r.name] = r
                    
                    _traverse(r, branch_index, visited)

        for branch_index, _input in enumerate(self.inputs, start=0):
            _traverse(root=_input, branch_index=branch_index, visited=set())

        return branches, multiple_branches

    def create_graph(self, branches):
        requires_output_from = OrderedDict()
        # requires_output_to = OrderedDict()
        layers = OrderedDict([(layer_name, layer) for branch in branches for layer_name, layer in branch.items()])
        for layer_name, layer in layers.items():
            required_outputs_from = [node.name for node in layer._node_container.inbound_nodes]
            requires_output_from[layer_name] = required_outputs_from
            # required_outputs_to = [node.name for node in layer._node_container.outbound_nodes]
            # requires_output_to[layer_name] = required_outputs_to

        params = tuple(layers[layer_name].params for layer_name in requires_output_from.keys())
        return layers, requires_output_from, params

    
    def update_params(self, new_params):
        for param, layer_name in zip(new_params, self.dependencies.keys()):
            self.layers[layer_name].update_weights(param)
    

    def call_with_external_weights(self, params, tensors):
        saved_outputs = OrderedDict()
        consumed_indices = 0
        for param, (key, val) in zip(params, self.dependencies.items()):
            if len(val) == 0:
                saved_outputs[key] = self.layers[key].call_with_external_weights(param, tensors[consumed_indices])
                consumed_indices += 1
            else:
                if len(val) == 1:
                    saved_outputs[key] = self.layers[key].call_with_external_weights(param, saved_outputs[val[0]])
                else:
                    saved_outputs[key] = self.layers[key].call_with_external_weights(param, [saved_outputs[v] for v in val])
        if self.multiple_branches:
            return [saved_outputs[layer.name] for layer in self.outputs]
        else:
            return saved_outputs[self.outputs[0].name]

    def __call__(self, inputs):
        return self.call_with_external_weights(self.params, inputs)


In [92]:
graph = GraphV2(input=inputs, outputs=[output, output2])

In [83]:
def loss(params, x, y):
    preds = graph.call_with_external_weights(params, [x])
    return jnp.mean((preds - y)**2)


In [84]:
grad_loss = jax.grad(loss, argnums=(0,))

In [94]:
graph.branches

[OrderedDict([('Input_1',
               <Input_1 Layer with input shape (28, 28, 1) and output shape (28, 28, 1)>),
              ('Conv2D_1',
               <Conv2D_1 Layer with input shape (1, 28, 28, 1) and output shape (1, 28, 28, 64)>),
              ('Activation_1',
               <Activation_1 Layer with input shape (1, 28, 28, 64) and output shape (1, 28, 28, 64)>),
              ('Conv2D_2',
               <Conv2D_2 Layer with input shape (1, 28, 28, 64) and output shape (1, 28, 28, 128)>),
              ('Activation_2',
               <Activation_2 Layer with input shape (1, 28, 28, 128) and output shape (1, 28, 28, 128)>),
              ('Conv2D_3',
               <Conv2D_3 Layer with input shape (1, 28, 28, 128) and output shape (1, 28, 28, 128)>),
              ('Flatten_1',
               <Flatten_1 Layer with input shape (1, 28, 28, 128) and output shape (100352,)>),
              ('Dense_1',
               <Dense_1 Layer with input shape (100352,) and output shape (Non

In [43]:
for layer in chain(branches[0]):
    print(layer)

[<Input_1 Layer with input shape (28, 28, 1) and output shape (28, 28, 1)>, <Conv2D_1 Layer with input shape (1, 28, 28, 1) and output shape (1, 28, 28, 64)>, <Activation_1 Layer with input shape (1, 28, 28, 64) and output shape (1, 28, 28, 64)>, <Conv2D_2 Layer with input shape (1, 28, 28, 64) and output shape (1, 28, 28, 128)>, <Activation_2 Layer with input shape (1, 28, 28, 128) and output shape (1, 28, 28, 128)>, <Conv2D_3 Layer with input shape (1, 28, 28, 128) and output shape (1, 28, 28, 128)>, <Add_1 Layer with input shape (1, 28, 28, 128) and output shape (1, 28, 28, 128)>]
[<Input_2 Layer with input shape (28, 28, 1) and output shape (28, 28, 1)>, <Conv2D_4 Layer with input shape (1, 28, 28, 1) and output shape (1, 28, 28, 32)>, <Activation_3 Layer with input shape (1, 28, 28, 32) and output shape (1, 28, 28, 32)>, <Conv2D_5 Layer with input shape (1, 28, 28, 32) and output shape (1, 28, 28, 128)>, <Activation_4 Layer with input shape (1, 28, 28, 128) and output shape (1, 28

In [4]:
model = Model(input=inputs, output=output)

In [5]:
model.compile(loss='binary_crossentropy', optimizer='sgd')

In [6]:
model.fit(train_x, train_y, epochs=30, batch_size=32)

 56%|██████████████████████▌                 | 18/32 [00:00<00:00, 22.71it/s, loss=0.03184072, remaining_epochs=7] 


KeyboardInterrupt: 